In [ ]:
import datetime
import os

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import seaborn as sns

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette(['#9e0059', '#6da7de', '#ee266d', '#dee000', '#eb861e'])
sns.set_context('paper', font_scale=1.3)    # Single-column figure.

In [ ]:
dir_name = '../doc/log/scale'
tool_names = ['GLEAMS', 'falcon', 'MaRaCluster', 'MS-Cluster',
              'spectra-cluster']

In [ ]:
resources = []
for tool in tool_names:
    for index in range(3):
        filename = os.path.join(dir_name, f'scale_{tool.lower()}_{index}.log')
        if os.path.isfile(filename):
            with open(filename) as f_in:
                for line in f_in:
                    if 'Elapsed (wall clock) time' in line:
                        time = pd.Timedelta(line[line.rfind(' ') + 1:].strip())
                        resources.append((tool, index, time))
                        break
        else:
            resources.append((tool, index, pd.Timedelta(0)))
resources = pd.DataFrame(resources, columns=['tool', 'index', 'time'])
resources['time_min'] = resources['time'].astype('timedelta64[m]')

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width, height))
with sns.color_palette(['#9e0059', '#ee266d', '#eb861e', '#6da7de', '#dee000']):
    sns.barplot(data=resources, x='index', y='time_min', hue='tool',
                hue_order=['GLEAMS', 'MaRaCluster', 'spectra-cluster',
                           'falcon', 'MS-Cluster'])

ax.legend(loc='lower center', bbox_to_anchor=(0.5, 1), ncol=2, frameon=False)

ax.set_xticklabels(['28 million', '56 million', '84 million'])
ax.set_xlabel('Number of spectra')
ax.set_ylabel('Runtime (minutes)')

sns.despine()

plt.savefig('cluster_comparison_resources.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()